In [2]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import pyodbc
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import gensim

In [ ]:
DMS = pd.read_csv('DMS.csv')
DMS.head()

In [ ]:
subDMS = DMS[['Discipline','Title','Original Source']]
subDMS['Combined'] = subDMS[['Title', 'Original Source']].apply(lambda x: ','.join(x.dropna()), axis=1)
subDMS

In [ ]:
def processText(text):
    text_tokens = word_tokenize(text)
    #text_tokens = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text)
    text_tokens = [word for word in text_tokens if not word in stop_words]  ##Remove Stop Words
    text_tokens = [ps.stem(word) for word in text_tokens] #Stem Words
    text_tokens = [word for word in text_tokens if len(word) > 2] #Removing noise
    text_tokens = [word for word in text_tokens if word.isalpha()] #Remove Punctuation
    text_tokens = [ele for ele in text_tokens if ele.strip()] #remove empty spaces in list
    return text_tokens

subDMS['Dataset'] = subDMS['Combined'].apply(processText)
subDMS

In [106]:
X = subDMS['Combined']
y = subDMS['Discipline']
#Spliting Data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=2)
#processing stop_words
count_vectorizer = CountVectorizer(stop_words = 'english')
#training "fit of curve" sort of deal
count_train = count_vectorizer.fit_transform(X_train.values)
#transforms our test data using the above fit.
count_test = count_vectorizer.transform(X_test.values)
print(count_vectorizer.get_feature_names()[1000:1002])
#Classifier for words
nb_classifier = MultinomialNB()
nb_classifier.fit(count_train,y_train)
pred = nb_classifier.predict(count_test)
print(metrics.accuracy_score(y_test,pred))


['36528', '36623']
0.8405260994656802


/home/spark/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
subDMS['predict'] = subDMS['Dataset'].apply(lambda x: (count_vectorizer.transform([ ' '.join(x)]))).apply(lambda x: nb_classifier.predict( x ))


In [ ]:
#Drwawings test
hostVM = r''
userVM = r''
pwdVM = ''
dbSQL = ''
sqlStatement=  """SELECT TOP (25) DrawingID, SUBSTRING(Name,9,100) as Name, DrawingTextAll
    FROM [Generation].[quest].[Drawings]
    WHERE DrawingTextAll IS NOT NULL AND Name LIKE '%Mill Creek%' AND DRAWINGID < \'3437674\' ORDER BY DrawingID DESC
    """
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+hostVM+';DATABASE='+dbSQL+';UID='+userVM+';PWD='+ pwdVM)
drawings = pd.read_sql(sqlStatement, cnxn)
drawings.head()

In [ ]:
drawings['set'] = drawings['Name'].apply(lambda x: x.replace(' ','-').split('-'))
drawings['predict'] = drawings['set'].apply(lambda x: (count_vectorizer.transform([ ' '.join(x)]))).apply(lambda x: nb_classifier.predict( x ))


In [ ]:
drawings